## Homework

In this homework, we'll deploy the Straight vs Curly Hair Type model we trained in the 
[previous homework](../08-deep-learning/homework.md).

Download the model from here: 

https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/model_2024_hairstyle.keras


In [ ]:
URL="https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/model_2024_hairstyle.keras"

!wget -P data {URL}
# !unzip data/data.zip

In [1]:
from tensorflow import lite
from tensorflow.keras.models import load_model
import os

2024-12-15 20:49:24.409226: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-15 20:49:24.460069: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-15 20:49:24.483877: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-15 20:49:24.489583: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-15 20:49:24.535344: I tensorflow/core/platform/cpu_feature_guar

In [2]:
model_dir = os.path.join('data', 'model_2024_hairstyle.keras')
model = load_model(model_dir)
model.summary()

I0000 00:00:1734306571.405913   96523 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734306571.430225   96523 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734306571.430259   96523 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734306571.432570   96523 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734306571.432605   96523 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 198, 198, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 313632)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │    20,072,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,146,948 (153.15 MB)

 Trainable params: 20,073,473 (76.57 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 20,073,475 (76.57 MB)

## Question 1

Now convert this model from Keras to TF-Lite format.

What's the size of the **converted** model?

* 27 Mb
* 43 Mb
* 77 Mb
* 127 Mb




In [3]:
converter = lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

tflite_dir = os.path.join('data', 'model_2024_hairstyle.tflite')
with open(tflite_dir, 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmprcuk_1i6/assets


INFO:tensorflow:Assets written to: /tmp/tmprcuk_1i6/assets


Saved artifact at '/tmp/tmprcuk_1i6'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  139645233928272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139645233929040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139645233929808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139645233930576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139645233931728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139645233932304: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1734306613.592649   96523 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1734306613.592736   96523 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-12-15 20:50:13.593460: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmprcuk_1i6
2024-12-15 20:50:13.593762: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-12-15 20:50:13.593767: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmprcuk_1i6
2024-12-15 20:50:13.596421: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-12-15 20:50:13.596844: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-12-15 20:50:13.646891: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmprcuk_1i6
2024-12-15 20:50:13.651244: I tensorflow/cc/saved_model/loader.cc

In [4]:
size_mb = os.path.getsize(tflite_dir) / (1024 * 1024)
print(size_mb)

76.5767936706543


## Question 2

To be able to use this model, we need to know the index of the input and 
the index of the output. 

What's the output index for this model?

* 3
* 7
* 13
* 24

In [6]:
interpreter = lite.Interpreter(model_path=tflite_dir)
interpreter.allocate_tensors()
output_index = interpreter.get_output_details()[0]['index']
print(output_index)

13


## Preparing the image

You'll need some code for downloading and resizing images. You can use 
this code:

```python
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img
```

For that, you'll need to have `pillow` installed:

```bash
pip install pillow
```

Let's download and resize this image: 

https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

Based on the previous homework, what should be the target size for the image?

**R: The target size should be (200, 200).**

In [8]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

## Question 3

Now we need to turn the image into numpy array and pre-process it. 

> Tip: Check the previous homework. What was the pre-processing 
> we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

* 0.24
* 0.44
* 0.64
* 0.84

In [9]:
img_url = 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'

raw_img = download_image(img_url)
img = prepare_image(raw_img, (200, 200))

In [10]:
import numpy as np

image_array = np.array(img, dtype=np.float32) / 255.0
print(image_array[0, 0, 0])

0.23921569


## Question 4

Now let's apply this model to this image. What's the output of the model?

* 0.293
* 0.493
* 0.693
* 0.893

In [11]:
interpreter.set_tensor(interpreter.get_input_details()[0]['index'], np.expand_dims(image_array, axis=0))
interpreter.invoke()
output = interpreter.get_tensor(interpreter.get_output_details()[0]['index'])[0][0]
print(output)

0.8934675


## Prepare the lambda code 

Now you need to copy all the code into a separate python file. You will 
need to use this file for the next two questions.

Tip: you can test this file locally with `ipython` or Jupyter Notebook 
by importing the file and invoking the function from this file.  


## Docker 

For the next two questions, we'll use a Docker image that we already 
prepared. This is the Dockerfile that we used for creating the image:

```docker
FROM public.ecr.aws/lambda/python:3.10

COPY model_2024_hairstyle_v2.tflite .

RUN pip install numpy==1.23.1
```

Note that it uses Python 3.10. The latest models of TF Lite
do not support Python 3.12 yet, so we need to use 3.10. Also,
for this part, we will use TensorFlow 2.14.0. We have tested
it, and the models created with 2.17 could be served with 2.14.0.

For that image, we also needed to use an older version of numpy
(1.23.1)

The docker image is published to [`agrigorev/model-2024-hairstyle:v3`](https://hub.docker.com/r/agrigorev/model-2024-hairstyle/tags).

A few notes:

* The image already contains a model and it's not the same model
  as the one we used for questions 1-4.
* The wheel for this combination that you'll need to use in your Docker image is https://github.com/alexeygrigorev/tflite-aws-lambda/raw/main/tflite/tflite_runtime-2.14.0-cp310-cp310-linux_x86_64.whl


## Question 5

Download the base image `agrigorev/model-2024-hairstyle:v3`. You can do it with [`docker pull`](https://docs.docker.com/engine/reference/commandline/pull/).

So what's the size of this base image?

* 182 Mb
* 382 Mb
* 582 Mb
* 782 Mb

You can get this information when running `docker images` - it'll be in the "SIZE" column.

In [13]:
!docker pull agrigorev/model-2024-hairstyle:v3

v3: Pulling from agrigorev/model-2024-hairstyle

124cce46: Pulling fs layer 
8b038848: Pulling fs layer 
8c0b7487: Pulling fs layer 
7e7c1be9: Pulling fs layer 
750232a5: Pulling fs layer 
68a79b8a: Pulling fs layer 
0580071d: Pulling fs layer 
68a79b8a: Download complete MB/104.9MBDigest: sha256:6d0aa19127467401fd439c76e662ceba2bd21a5713f174d4f89b354a26939ea6
Status: Downloaded newer image for agrigorev/model-2024-hairstyle:v3
docker.io/agrigorev/model-2024-hairstyle:v3


In [14]:
!docker images

REPOSITORY                       TAG       IMAGE ID       CREATED       SIZE
agrigorev/model-2024-hairstyle   v3        6d0aa1912746   12 days ago   1.14GB


## Question 6

Now let's extend this docker image, install all the required libraries
and add the code for lambda.

You don't need to include the model in the image. It's already included. 
The name of the file with the model is `model_2024_hairstyle_v2.tflite` and it's 
in the current workdir in the image (see the Dockerfile above for the 
reference). 
The provided model requires the same preprocessing for images regarding target size and rescaling the value range than used in homework 8.

Now run the container locally.

Score this image: https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

What's the output from the model?

* 0.229
* 0.429
* 0.629
* 0.829


In [16]:
!curl -X POST http://localhost:9000/2015-03-31/functions/function/invocations \
    -d '{"url": "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"}'

0.42985352873802185

## Optional - AWS and Docker

### Publishing it to AWS

Now you can deploy your model to AWS!

* Publish your image to ECR
* Create a lambda function in AWS, use the ECR image
* Give it more RAM and increase the timeout 
* Test it
* Expose the lambda function using API Gateway

This is optional and not graded.


### Publishing to Docker hub

Just for the reference, this is how we published our image to Docker hub:

```bash
docker build -t model-2024-hairstyle -f homework.dockerfile .
docker tag model-2024-hairstyle:latest agrigorev/model-2024-hairstyle:v3
docker push agrigorev/model-2024-hairstyle:v3
```

(You don't need to execute this code)